### Read in the data file

Note that this is a very large data set with over 3.5 Gb. However, we will select
only fake and reliable news from this data set to analyze.

First we will work with a small chunk of data.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

import nltk
import re, string, unicodedata
import contractions
import inflect
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

### Pipeline

Goal: Transform each Content columns and Title into a multiple vectors.

Cleaning
1. De-noise
2. Tokenize
3. Normalize

Feature Engineering
1. Bag of Words
2. TF-IDF
3. Word Embedding

Further Improvement
1. POS tagging
2. Sentiment analysis
3. Using vec2word somehow!

Deep Learning
1. Layers config
2. loss config
3. optimizer config

In [6]:
# Clean all columns and tokenize
# df.content = df.content.map(denoise_text)
# df.content = df.content.map(nltk.word_tokenize)
# df.content = df.content.map(normalize)
#
# df.head()

### Read and clean each chunk

In [7]:
# Increase csv file limit
import csv
csv.field_size_limit(2000000)

# Data path for all saved data
data_path = 'D:\\PycharmProjects\\springboard\\data\\'
file_name = 'news_cleaned_2018_02_13.csv'

# Chunk size
chunksize = 5000

# Only extract content and type columns
use_cols = ['type', 'content']
use_rows = ['fake', 'reliable']

# read data in chunk
data_iterator = pd.read_csv(f'{data_path}{file_name}',
                    usecols=use_cols,
                    chunksize=chunksize,
                    encoding='ISO-8859-1',
                    index_col=False,
                    engine='python')

chunk_list = []
for chunk in data_iterator:
    filtered_chunk = chunk[chunk.type.isin(use_rows)]
    chunk_list.append(filtered_chunk)

filtered_data = pd.concat(chunk_list)

In [11]:
filtered_data.to_csv("D:\\PycharmProjects\\springboard\\data\\news_clean_1.csv")
